In [1]:
# useing this variable for saving models and summaries
exp_name = "CNN-LSTM4001"

## Running CNN then RNN
5 layers CNN [2, 2, 2, 2, 2], then RNN, bias_shape = [32,64,64, 64, 64]

In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import tensorflow as tf
import sys
from hdf5storage import savemat
import pickle

sys.path.append('../src')
sys.path.append('../src/models')

In [3]:
import models.CNN_lstm_model as CNN_lstm_model
import training  
import evaluation.nn_eval as nn_eval 

## Loading synthetic data

In [4]:
#loading data
data_path= '/data2/data/zebrahim/synthetic_dataset/data_8192_1000_3_4_4_.03_.03_.2' + 'processed_data.p'

with open(data_path, 'r') as fin:
    data = pickle.load(fin)
    
train_data = data['train_data']
validation_data = data['validation_data']
test_data = data['test_data']

train_gt = np.expand_dims(data['train_gt'], axis=-1)
validation_gt = np.expand_dims(data['validation_gt'], axis=-1)
test_gt = np.expand_dims(data['test_gt'], axis=-1)

index_of_changes = data['index_of_changes']
 

## Model

In [5]:
## Model Parameter
n_variables= 12
learning_rate = 0.001
batch_size = 10

pooling_stride = [2, 2, 2, 2, 2]
weights_shape = [[5,n_variables, 32],
                 [5, 32, 64],
                 [5, 64, 64],
                 [5, 64, 64],
                 [5, 64, 64]]

bias_shape = [32,64,64, 64, 64]
activation = tf.nn.relu

lstm_n_hidden = 128
n_classes = 1


#positive weight for weighted cross entropy
s = train_data.shape
resolution = np.prod(pooling_stride)
gt_effective = np.reshape(train_gt, (s[0]*(s[1]/resolution), resolution))
pos_weight = (1.0 - gt_effective.max(axis=1).mean()) / gt_effective.max(axis=1).mean()

In [6]:
pos_weight

3.186018221237171

In [7]:
# Launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [8]:
summary_path = '../summary/' + exp_name 
checkpoint_path = '../model/' + exp_name + '/' + exp_name 


In [9]:
model = CNN_lstm_model.Model(n_variables, n_classes)

In [10]:
model.build_cnn_lstm_net(pooling_stride,
                         weights_shape,
                         bias_shape,
                         activation,
                         lstm_n_hidden,
                         n_classes,
                         learning_rate,
                         pos_weight)

In [11]:
train_obj = training.Trainer(model,
                 train_data, train_gt,
                 validation_data, validation_gt,
                 sess,
                 summary_path,
                 batch_size,
                 global_step = 0)

In [12]:
train_obj.train(max_iter = 10000,
                train_eval_step=100,
                validation_eval_step=100,
                display_step=100)

Tarin Iter 0, auc= 0.279903, fmeasure_max= 0.399622, precision= 0.249705, recall= 1.000000
Train Random guess:  , auc= 0.252671fmeasure_max= 0.397347, precision= 0.248420, recall= 0.992114
Validation Iter 0, auc= 0.201578, fmeasure_max= 0.376737, precision= 0.232086, recall= 1.000000
Validation Random guess:  auc= 0.222127, fmeasure_max= 0.371624, precision= 0.229317, recall= 0.979417
Minibatch Loss= 1.128882
Tarin Iter 100, auc= 0.667800, fmeasure_max= 0.639157, precision= 0.626506, recall= 0.652330
Train Random guess:  , auc= 0.228090fmeasure_max= 0.359827, precision= 0.220906, recall= 0.969534
Validation Iter 100, auc= 0.588794, fmeasure_max= 0.604918, precision= 0.579278, recall= 0.632933
Validation Random guess:  auc= 0.230111, fmeasure_max= 0.371750, precision= 0.228673, recall= 0.993139
Minibatch Loss= 0.819386
Tarin Iter 200, auc= 0.656517, fmeasure_max= 0.667206, precision= 0.685524, recall= 0.649842
Train Random guess:  , auc= 0.246543fmeasure_max= 0.399361, precision= 0.2504

In [13]:
saver = tf.train.Saver()
saver.save(sess, checkpoint_path, global_step=train_obj.global_step)

'../model/CNN-LSTM4001/CNN-LSTM4001-10000'

In [ ]:
saver = tf.train.Saver()
saver.restore(sess, checkpoint_path+'-1000')

In [3]:
probability_of_pos = nn_eval.evaluate_model(model,
                                   sess,
                                   test_data,
                                   test_gt)

#%85

NameError: name 'nn_eval' is not defined

In [15]:
#shape of probability_change is (batch size, T, num_classes)
#since num_classes = 1 we squeeze that axis
probability_change = np.squeeze(probability_of_pos, axis = 2)

In [16]:
savemat('../poc/poc'+exp_name+'.mat', {'poc':probability_change})

In [1]:
x = sum(n/2 for n in range(2,6,2))

In [2]:
x

3